##### <a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/STT%20JVS%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
!sudo fusermount -u ~/general-304503

In [120]:
!mkdir ~/general-304503
!gcsfuse general-304503 ~/general-304503

mkdir: cannot create directory ‘/home/jsjsrobert500/general-304503’: File exists
Using mount point: /home/jsjsrobert500/general-304503
2021/03/16 01:42:10.768570 Opening GCS connection...
2021/03/16 01:42:10.945469 Mounting file system...
2021/03/16 01:42:10.948109 File system has been successfully mounted.


In [121]:
!pip install --user tfds-nightly
!pip install --user pydub
!pip install --user tensorflow_io

In [122]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import time
import tensorflow_io as tfio

In [123]:
!git clone https://github.com/Jaidon-Smith/public_datasets.git

fatal: destination path 'public_datasets' already exists and is not an empty directory.


# Setting Up Tensorboard

In [124]:
from datetime import datetime

In [125]:
# Initialise Logs

!rm -r ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21/logs

%mkdir ~/general-304503/notebook_logs/
%mkdir ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21
%mkdir ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21/logs

mkdir: cannot create directory ‘/home/jsjsrobert500/general-304503/notebook_logs/’: File exists
mkdir: cannot create directory ‘/home/jsjsrobert500/general-304503/notebook_logs/jvs_transformer_stt_12_3_21’: File exists


In [126]:
# Initialise Checkpoints
!rm -r ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21/checkpoints

%mkdir ~/general-304503/notebook_logs/
%mkdir ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21
%mkdir ~/general-304503/notebook_logs/jvs_transformer_stt_12_3_21/checkpoints

mkdir: cannot create directory ‘/home/jsjsrobert500/general-304503/notebook_logs/’: File exists
mkdir: cannot create directory ‘/home/jsjsrobert500/general-304503/notebook_logs/jvs_transformer_stt_12_3_21’: File exists


In [127]:
logdir = 'gs://general-304503/notebook_logs/jvs_transformer_stt_12_3_21/logs'
checkpoint_path = 'gs://general-304503/notebook_logs/jvs_transformer_stt_12_3_21/checkpoints'

In [128]:
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Building the Model

In [129]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1treUWLprQkcGOXeiFPofOvtk4iPZhsJq' -O hiragana_jsut.model

--2021-03-16 01:42:20--  https://docs.google.com/uc?export=download&id=1treUWLprQkcGOXeiFPofOvtk4iPZhsJq
Resolving docs.google.com (docs.google.com)... 108.177.120.113, 108.177.120.102, 108.177.120.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.120.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2inik6330hh37q5hl5iiv59i8j5m1h73/1615858875000/04186398190322129029/*/1treUWLprQkcGOXeiFPofOvtk4iPZhsJq?e=download [following]
--2021-03-16 01:42:20--  https://doc-0g-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2inik6330hh37q5hl5iiv59i8j5m1h73/1615858875000/04186398190322129029/*/1treUWLprQkcGOXeiFPofOvtk4iPZhsJq?e=download
Resolving doc-0g-28-docs.googleusercontent.com (doc-0g-28-docs.googleusercontent.com)... 173.194.195.132, 2607:f8b0:4001:c11::84
Connecting to doc-0g-28-docs.googleusercontent.com (doc-0g

In [130]:
!pip install --quiet tensorflow-text

import tensorflow_text as text
from tensorflow.python.platform import gfile

model_file = 'hiragana_jsut.model'
model = gfile.GFile(model_file, 'rb').read()

tokenizer = text.SentencepieceTokenizer(model=model)

input_vocab_size = tokenizer.vocab_size().numpy()
print(input_vocab_size)

2000


In [131]:
!pip install --user pykakasi

In [132]:
import pykakasi
converter = pykakasi.kakasi()

In [133]:
def preprocess_text(old_text):
  text = old_text
  text = tokenizer.tokenize(text)
  text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  return text

In [134]:
def preprocess_text_no_pad(old_text):
  text = old_text
  text = tokenizer.tokenize(text)
  text = tf.shape(text)[0]
  return text

In [135]:
def preprocess_audio(old_audio, original_sample_rate):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, original_sample_rate, 24000)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)
  audio = tf.pad(audio, paddings=[[0 , 500 - tf.shape(audio)[0]], [0,0]])
  return audio

In [136]:
def preprocess_audio_no_pad(old_audio, original_sample_rate):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, original_sample_rate, 24000)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)
  audio = tf.shape(audio)[0]
  return audio

In [137]:
download_config = tfds.download.DownloadConfig(manual_dir='gs://general-304503/public_datasets/downloads/manual')

jsut_dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='gs://general-304503/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [138]:
def convert_to_kana(text):
  #kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
  new_text = text.numpy().decode('utf-8')
  kana = converter.convert(new_text)
  return ''.join([i['hira'] for i in kana])

In [139]:
jsut_dataset = jsut_dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": tf.py_function(convert_to_kana, [x["text"]], Tout=tf.string)})

In [140]:
jsut_dataset = jsut_dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"], 48000),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"], 48000),
                                 "text": tf.py_function(preprocess_text, [x["text"]], Tout=tf.int32),
                                 "text_lengths": tf.py_function(preprocess_text_no_pad, [x["text"]], Tout=tf.int32)})

In [141]:
jsut_dataset = jsut_dataset.batch(8)
jsut_dataset = jsut_dataset.prefetch(tf.data.AUTOTUNE)

## jvs dataset

In [142]:
download_config = tfds.download.DownloadConfig(manual_dir='gs://general-304503/public_datasets/downloads/manual')

jvs_dataset, info = tfds.load(
                    "jvs",
                    split="train",
                    data_dir='gs://general-304503/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [143]:
jvs_dataset = jvs_dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": tf.py_function(convert_to_kana, [x["text"]], Tout=tf.string)})

In [144]:
jvs_dataset = jvs_dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"], 48000),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"], 48000),
                                 "text": tf.py_function(preprocess_text, [x["text"]], Tout=tf.int32),
                                 "text_lengths": tf.py_function(preprocess_text_no_pad, [x["text"]], Tout=tf.int32)})

In [145]:
jvs_dataset = jvs_dataset.batch(8)
jvs_dataset = jvs_dataset.prefetch(tf.data.AUTOTUNE)



```python
ds = jvs_dataset.take(1)

for i in ds:
  data = i
  break

speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']

tokenizer.detokenize(text.numpy()[1]).numpy().decode('utf-8')
```



In [146]:
input_vocab_size = tokenizer.vocab_size()
print(input_vocab_size)

tf.Tensor(2000, shape=(), dtype=int32)


# Using Transformer Encoder Blocks

In [147]:
d_model = 512

In [148]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [149]:
def scaled_dot_product_attention(q, k, v):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)


  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [150]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [151]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [152]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training):

    attn_output, _ = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [153]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(d_model)

    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training=True):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # Makes input larger by a constant
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training)
    
    return x  # (batch_size, input_seq_len, d_model)

In [154]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [155]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [156]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='validation_loss')
instant_loss = tf.keras.metrics.Mean(name='instant_loss')

In [157]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding):
    super(Transformer, self).__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           maximum_position_encoding)
    self.final_layer = tf.keras.layers.Dense(input_vocab_size)

  def call(self, inp, training=True):
    enc_output = self.encoder(inp, training)  # (batch_size, inp_seq_len, d_model)
    final_output = self.final_layer(enc_output)  # (batch_size, tar_seq_len, target_vocab_size)
    return final_output

transformer = Transformer(num_layers=6,
          d_model=d_model,
          num_heads=8,
          dff=2048, 
          maximum_position_encoding=10000)

In [158]:
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [159]:
@tf.function
def train_step(speech, text, speech_lengths, text_lengths):
  with tf.GradientTape() as tape:
    logits = transformer(speech)

    labels = text
    label_length = text_lengths
    logit_length = speech_lengths
    unique = tf.nn.ctc_unique_labels(text)


    loss = tf.nn.ctc_loss(
        labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
    )

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  instant_loss(loss)

In [160]:
@tf.function
def validation_step(speech, text, speech_lengths, text_lengths):
  logits = transformer(speech)

  labels = text
  label_length = text_lengths
  logit_length = speech_lengths
  unique = tf.nn.ctc_unique_labels(text)


  loss = tf.nn.ctc_loss(
      labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
  )


  validation_loss(loss)

In [161]:
EPOCHS = 5
validate_period = 30
save_period = 200
print_period = 1
train_period = 30

In [162]:
dataset = jvs_dataset

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  validation_loss.reset_states()
  instant_loss.reset_states()

  jsut_dataset.take(5)
  # inp -> portuguese, tar -> english
  for (batch, data) in enumerate(jvs_dataset):
    #print(batch)
    #if batch == 9:
    #  print(tokenizer.detokenize(data['text']).numpy()[0].decode('utf-8'))
    #  print(data['text'].numpy()[0])
    #  continue

    train_step(data['speech'], data['text'], data['speech_lengths'], data['text_lengths'])


    if (batch+1) % print_period == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {instant_loss.result():.4f}')
      instant_loss.reset_states()

    if (batch+1) % train_period == 0:
      with file_writer.as_default():
        tf.summary.scalar("train_loss", train_loss.result(), step=batch)
      print(f'Training Print: Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f}')
      train_loss.reset_states()

    if (batch + 1) % save_period == 0:
      ckpt_save_path = ckpt_manager.save()
      print (f'Saving checkpoint for batch {batch} at {ckpt_save_path}')
    if (batch + 1) % validate_period == 0:
      for (val_batch, val_data) in enumerate(jsut_dataset.take(5)):
        validation_step(val_data['speech'], val_data['text'], val_data['speech_lengths'], val_data['text_lengths'])
      with file_writer.as_default():
        tf.summary.scalar("validation_loss", validation_loss.result(), step=batch)
      print(f'Validation Print: Epoch {epoch + 1} Batch {batch} Loss {validation_loss.result():.4f}')
      validation_loss.reset_states()

  break
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print (f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


Epoch 1 Batch 0 Loss 568.7057
Epoch 1 Batch 1 Loss 534.1376
Epoch 1 Batch 2 Loss 581.6332
Epoch 1 Batch 3 Loss 540.1230
Epoch 1 Batch 4 Loss 580.5500
Epoch 1 Batch 5 Loss 715.5375
Epoch 1 Batch 6 Loss 432.8978
Epoch 1 Batch 7 Loss 491.5380
Epoch 1 Batch 8 Loss 648.2237
Epoch 1 Batch 9 Loss 499.2842
Epoch 1 Batch 10 Loss 469.4774
Epoch 1 Batch 11 Loss 471.4719
Epoch 1 Batch 12 Loss 632.8846
Epoch 1 Batch 13 Loss 475.6606
Epoch 1 Batch 14 Loss 661.6863
Epoch 1 Batch 15 Loss 442.8964
Epoch 1 Batch 16 Loss 539.0316
Epoch 1 Batch 17 Loss 455.4348
Epoch 1 Batch 18 Loss 529.8572
Epoch 1 Batch 19 Loss 426.7677
Epoch 1 Batch 20 Loss 354.0236
Epoch 1 Batch 21 Loss 559.0055
Epoch 1 Batch 22 Loss 359.3776
Epoch 1 Batch 23 Loss 453.7595
Epoch 1 Batch 24 Loss 473.1254
Epoch 1 Batch 25 Loss 438.0790
Epoch 1 Batch 26 Loss 381.3424
Epoch 1 Batch 27 Loss 417.4186
Epoch 1 Batch 28 Loss 335.9699
Epoch 1 Batch 29 Loss 334.9075
Training Print: Epoch 1 Batch 29 Loss 493.4936
Validation Print: Epoch 1 Batch 2